## 라이브러리

In [ ]:
import pandas as pd
import numpy as np

## 데이터 보기

In [45]:
df = pd.read_table('./price_column_added.txt',sep=',') ## 경로는 각자 상황에 맞게 넣어주세요

In [4]:
df.head()

,Unnamed: 0,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1,PRICE
0,0,2858,2014-01-07,AA,1892,A,2020-01-01,1,GN,9018.160
1,1,5647,2014-02-14,BB,182009,A,2020-01-01,1,J,9018.160
2,2,33314,2014-11-20,BB,82431,A,2020-01-01,1,SC,9018.160
3,3,37001,2014-12-04,BB,725,C,2020-01-01,1,MP,7376.125
4,4,37819,2014-12-07,AA,220691,C,2020-01-01,1,JRR,7376.125


In [2]:
## 칼럼보기

for col in df.columns:
    print(col)

Unnamed: 0
USER_ID
JOIN_DATE
D_TYPE
STORE_ID
GOODS_TYPE
DATE
COUNT
AD1
PRICE


## 데이터 전처리

In [10]:
## 문자열로된 날짜를 Timestamp형식으로 변환
df['JOIN_DATE'] = pd.to_datetime(df['JOIN_DATE'])
df['DATE'] = pd.to_datetime(df['DATE'])

In [6]:
## Unnamed: 0 컬럼 drop

df = df.drop("Unnamed: 0", axis=1)

In [47]:
## 데이터 크기와 결측치 확인

print('행 개수 : ',len(df)) ## 데이터 개수
df.isnull().sum() ## 결측치 확인

행 개수 :  879271


Unnamed: 0    0
USER_ID       0
JOIN_DATE     0
D_TYPE        0
STORE_ID      0
GOODS_TYPE    0
DATE          0
COUNT         0
AD1           0
PRICE         0
dtype: int64

## 데이터 분석

In [12]:
from tqdm import tqdm
 
customer_id = list(df['USER_ID'].unique()) ## 유저아이디
 
## 먼저 각 유저아이디별로 구매금액이 얼마인지 알아보자.
monetary_df = pd.DataFrame()          ## 구매금액 데이터 초기화
monetary_df['USER_ID'] = customer_id  ## 유저아이디 삽입
 
monetary_data = []  ## 구매금액을 담을 리스트
for ci in tqdm(customer_id,position=0,desc='Calculating amount of individual customer'):
    temp = df.query('USER_ID==@ci')             ## 해당 아이디의 유저데이터 추출
    amount = sum(temp['COUNT'] * temp['PRICE']) ## 해당 유저 구매금액
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data  ## 구매금액 데이터 삽입
 
## 각 유저별 최근방문일을 알아보자.
temp_recency_df = df[['USER_ID','DATE']].drop_duplicates()      ## 유저 아이디와 결제날짜만 추출한뒤 중복 제거
recency_df = temp_recency_df.groupby('USER_ID')['DATE'].max().reset_index() ## 아이디로 그룹화 한다음 최근방문일을 구해야 하므로 결제날짜에 max를 적용한다.
recency_df = recency_df.rename(columns={'DATE':'Recency'})
 
## 각 유저별 방문횟수를 알아보자.
frequency_df = df.groupby('USER_ID')['DATE'].count().reset_index() ## 아이디로 그룹화 한다음 방문횟수를 구해야 한다. 여기서는 방문횟수를 결제날짜 개수로 생각했으므로 결제날짜에 count를 적용한다.
frequency_df = frequency_df.rename(columns={'DATE':'Frequency'})
 
## 데이터를 유저아이디를 기준으로 합쳐야한다.
rfm_df = pd.merge(recency_df,frequency_df,how='left',on='USER_ID')
rfm_df = pd.merge(rfm_df,monetary_df,how='left',on='USER_ID')

Calculating amount of individual customer: 100%|█| 165425/165425 [33:21<00:00, 82.64it/s] 


In [13]:
## RFM 데이터프레임 확인

rfm_df

,USER_ID,Recency,Frequency,Monetary
0,224,2020-12-09,2,16394.285
1,232,2020-11-18,23,190537.495
2,342,2020-10-31,1,9018.160
3,565,2020-11-12,2,8558.325
4,729,2020-09-05,1,9018.160
...,...,...,...,...
165420,1830551,2020-12-31,1,1182.200
165421,1830570,2020-12-31,1,1182.200
165422,1830580,2020-12-31,1,1182.200
165423,1830589,2020-12-31,1,1182.200


In [14]:
## 데이터 변환(날짜를 수치로 표현하기 위한)

current_day = pd.to_datetime('20200101')## 기준 날짜는 2020년 1월 1일
time_diff = rfm_df['Recency']-current_day ## 최근방문일과 기준 날짜의 시간 차이

In [15]:
time_diff

0        343 days
1        322 days
2        304 days
3        316 days
4        248 days
           ...   
165420   365 days
165421   365 days
165422   365 days
165423   365 days
165424   365 days
Name: Recency, Length: 165425, dtype: timedelta64[ns]

In [18]:
time_in_seconds = [x.total_seconds() for x in time_diff]  ## 시간 차이를 초단위로 계산
rfm_df['Recency'] = time_in_seconds   ## 변환된 데이터를 다시 삽입

### 최종 RFM 데이터프레임

In [19]:
rfm_df

,USER_ID,Recency,Frequency,Monetary
0,224,29635200.0,2,16394.285
1,232,27820800.0,23,190537.495
2,342,26265600.0,1,9018.160
3,565,27302400.0,2,8558.325
4,729,21427200.0,1,9018.160
...,...,...,...,...
165420,1830551,31536000.0,1,1182.200
165421,1830570,31536000.0,1,1182.200
165422,1830580,31536000.0,1,1182.200
165423,1830589,31536000.0,1,1182.200


### 점수 매기기 위한 구간 설정

In [26]:
## Recency 구분값(5구간) - (최소,최대값 기준)

Recency_list = []
for x in np.linspace(np.min(rfm_df['Recency']),np.max(rfm_df['Recency']),6)[1:-1]:
    Recency_list.append(x)
    print(f'구분값 : {x}')

구분값 : 6307200.0
구분값 : 12614400.0
구분값 : 18921600.0
구분값 : 25228800.0


In [44]:
## Recency 구분값(5구간) - (분위수 기준)
 
Recency_list2 = [] ## 분위수를 담을 리스트
for ql in np.linspace(0,1,6)[1:-1]:
    Recency_list2.append(np.quantile(rfm_df['Recency'],ql))
Recency_list2

[11145600.0, 18748800.0, 25056000.0, 28512000.0]

In [27]:
## Frequency 구분값(5구간) - (최소,최대값 기준)

Frequency_list = []
for x in np.linspace(np.min(rfm_df['Frequency']),np.max(rfm_df['Frequency']),6)[1:-1]:
    Frequency_list.append(x)
    print(f'구분값 : {x}')

구분값 : 74.0
구분값 : 147.0
구분값 : 220.0
구분값 : 293.0


In [49]:
## Frequency 구분값(5구간) - (분위수 기준)
 
Frequency_list2 = [] ## 분위수를 담을 리스트
for ql in np.linspace(0,1,6)[1:-1]:
    Frequency_list2.append(np.quantile(rfm_df['Frequency'],ql))
Frequency_list2

[1.0, 1.0, 2.0, 6.0]

In [28]:
## Monetary 구분값(5구간) - (최소,최대값 기준)

Monetary_list = []
for x in np.linspace(np.min(rfm_df['Monetary']),np.max(rfm_df['Monetary']),6)[1:-1]:
    Monetary_list.append(x)
    print(f'구분값 : {x}')

구분값 : 21923954.536
구분값 : 43846726.872
구분값 : 65769499.208000004
구분값 : 87692271.544


In [50]:
## Monetary 구분값(5구간) - (분위수 기준)
 
Monetary_list2 = [] ## 분위수를 담을 리스트
for ql in np.linspace(0,1,6)[1:-1]:
    Monetary_list2.append(np.quantile(rfm_df['Monetary'],ql))
Monetary_list2

[7376.125, 9018.16, 18036.32, 54108.96000000001]

## 데이터에 점수를 매겨주는 함수

In [51]:
def get_score(level, data):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue 
    return score
    
def get_rfm_grade(df, num_class, rfm_tick_point, rfm_col_map, suffix=None):
    '''
    Description :
    개별 고객에 대한 최근방문일/방문횟수/구매금액 데이터가 주어졌을때
    최근방문일/방문횟수/구매금액 점수를 계산하여 주어진 데이터 오른쪽에 붙여줍니다.
    
    Parameters :
    df = pandas.DataFrame 데이터
    num_class = 등급(점수) 개수
    rfm_tick_point = 최근방문일/방문횟수/구매금액에 대해서 등급을 나눌 기준이 되는 값
                    'quantile', 'min_max' 또는 리스트를 통하여 직접 값을 정할 수 있음.
                    단, 리스트 사용시 원소의 개수는 반드시 num_class - 1 이어야함.
                    quatile = 데이터의 분위수를 기준으로 점수를 매김
                    min_max = 데이터의 최소값과 최대값을 동일 간격으로 나누어 점수를 매김
    rfm_col_map = 최근방문일/방문횟수/구매금액에 대응하는 칼럼명
    예 - {'R':'Recency','F':'Frequency','M':'Monetary'}
    suffix = 최근방문일/방문횟수/구매금액에 대응하는 칼럼명 뒤에 붙는 접미사
    Return : 
    pandas.DataFrame
    '''
    ##### 필요모듈 체크
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    
    ##### 파라미터 체크
    if not isinstance(df, pd.DataFrame): ## 데이터는 pd.DataFrame이어야 함.
        print('데이터는 pandas.DataFrame 객체여야 합니다.')
        return
    
    if isinstance(rfm_tick_point, dict) == False or isinstance(rfm_col_map, dict) == False: ## rfm_tick_point와 rfm_col_map은 모두 딕셔너리
        print(f'rfm_tick_point와 rfm_col_map은 모두 딕셔너리여야합니다.')
        return
    
    if len(rfm_col_map) != 3: ## rfm_col_map에는 반드시 3개의 키를 가져아함.
        print(f'rfm_col_map인자는 반드시 3개의 키를 가져야합니다. \n현재 rfm_col_map에는 {len(rfm_col_map)}개의 키가 있습니다.')
        return
    
    if len(rfm_tick_point) != 3: ## rfm_tick_point에는 반드시 3개의 키를 가져아함.
        print(f'rfm_tick_point인자는 반드시 3개의 키를 가져야합니다. \n현재 rfm_col_map에는 {len(rfm_col_map)}개의 키가 있습니다.')
        return
    
    if set(rfm_tick_point.keys()) != set(rfm_col_map.keys()): ## rfm_tick_point와 rfm_col_map은 같은 키를 가져야함.
        print(f'rfm_tick_point와 rfm_col_map은 같은 키를 가져야 합니다.')
        return
    
    if not set(rfm_col_map.values()).issubset(set(df.columns)):
        not_in_df = set(rfm_col_map.values())-set(df.columns)
        print(f'{not_in_df}이 데이터 칼럼에 있어야 합니다.')
        return
    
    for k, v in rfm_tick_point.items():
        if isinstance(v, str):
            if not v in ['quantile','min_max']:
                print(f'{k}의 값은 "quantile" 또는 "min_max"중에 하나여야 합니다.')
                return
        elif isinstance(v,list) or isinstance(v,tuple):
            if len(v) != num_class-1:
                print(f'{k}에 대응하는 리스트(튜플)의 원소는 {num_class-1}개여야 합니다.')
                return
    
    if suffix:
        if not isinstance(suffix, str):
            print('suffix인자는 문자열이어야합니다.')
            return
        
    ##### 최근방문일/방문횟수/구매금액 점수 부여
    for k, v in rfm_tick_point.items():
        if isinstance(v,str):
            if v == 'quantile':
                ## 데이터 변환
                scale = preprocessing.StandardScaler() ## 데이터의 범위 조작하기 쉽게 해주는 클래스 
                temp_data = np.array(df[rfm_col_map[k]]) ## 데이터를 Numpy 배열로 변환
                temp_data = temp_data.reshape((-1,1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
                temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
                temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환
 
                ## 분위수 벡터
                quantiles_level = np.linspace(0,1,num_class+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data,ql)) ## 분위수를 계산하고 리스트에 삽입
            else: ## min_max인 경우
                ## 데이터 변환
                temp_data = np.array(df[rfm_col_map[k]])
 
                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data),np.max(temp_data),num_class+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점
        else: ## 직접 구분값을 넣어주는 경우
            temp_data = np.array(df[rfm_col_map[k]])
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = rfm_col_map[k]+'_'+k ## 점수값을 담는 변수의 이름
        if suffix:
            new_col_name = rfm_col_map[k]+'_'+suffix
        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [52]:
rfm_tick_point={'R':'quantile','F':'quantile','M':'quantile'}
rfm_col_map={'R':'Recency','F':'Frequency','M':'Monetary'}
 
result = get_rfm_grade(df=rfm_df, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)
result.to_csv('주차장_RFM_result.csv',index=False)